In [1]:
import pandas as pd
df_collection = {}

dfs_may = pd.read_html('stock_data/May_2000.htm', header =0)
df_collection['May'] = pd.DataFrame(dfs_may[0])
dfs_june = pd.read_html('stock_data/June_2000.htm', header =0)
df_collection['June'] = dfs_june[0]
dfs_july = pd.read_html('stock_data/July_2000.htm', header =0)
df_collection['July'] = dfs_july[0]
dfs_august = pd.read_html('stock_data/August_2000.htm', header =0)
df_collection['Aug'] = dfs_august[0]
dfs_sept = pd.read_html('stock_data/Sept_2000.htm', header =0)
df_collection['Sept'] = dfs_sept[0]
dfs_oct = pd.read_html('stock_data/Oct_2000.htm', header =0)
df_collection['Oct'] = dfs_oct[0]

print((df_collection['May']))

        Date Contract  Units  $Volume  LowPrice  HighPrice  AvgPrice  \
0   05/01/00      Dem    224  112.043     0.490      0.550     0.500   
1   05/01/00   Reform      2    0.067     0.019      0.048     0.034   
2   05/01/00      Rep    116   57.950     0.488      0.501     0.500   
3   05/02/00      Dem     87   44.369     0.501      0.522     0.510   
4   05/02/00   Reform     50    0.196     0.003      0.005     0.004   
..       ...      ...    ...      ...       ...        ...       ...   
88  05/30/00   Reform     53    0.318     0.006      0.006     0.006   
89  05/30/00      Rep    249  126.944     0.504      0.510     0.510   
90  05/31/00      Dem    252  126.890     0.492      0.514     0.504   
91  05/31/00   Reform    115    0.445     0.003      0.004     0.004   
92  05/31/00      Rep     67   33.803     0.495      0.507     0.505   

    LastPrice  
0       0.550  
1       0.019  
2       0.500  
3       0.508  
4       0.003  
..        ...  
88      0.006  
89     

In [2]:
for key in df_collection:
    df = df_collection[key]
    df_collection[key] = df_collection[key][df_collection[key].Contract != 'Reform']
    df_collection[key] = df_collection[key].drop(['Units','$Volume','LowPrice','HighPrice','LastPrice'], axis = 1)
    df_collection[key] = df_collection[key].dropna()
    df_collection[key]['NormalizedPrice'] = 0
    
    for index,row in df_collection[key].iterrows():
        if(row['Contract'] == 'Dem'):
            demPrice = float(row['AvgPrice'])
            #print(demPrice)
            repPrice = df_collection[key].loc[df_collection[key]['Date'] == row['Date']]
            repPrice = repPrice[repPrice.Contract != 'Dem']
            repPrice.reset_index(drop=True, inplace=True)
            print(repPrice)
            if(repPrice.empty):
                df_collection[key].loc[index,'NormalizedPrice'] = -1
            else:
                repPrice = repPrice.iloc[0]['AvgPrice']
                repPrice = float(repPrice)
                df_collection[key].loc[index,'NormalizedPrice'] = demPrice / (demPrice+repPrice)
        if(row['Contract'] == 'Rep'):
            repPrice = float(row['AvgPrice'])
            #print(demPrice)
            demPrice = df_collection[key].loc[df_collection[key]['Date'] == row['Date']]
            demPrice = demPrice[demPrice.Contract != 'Rep']
            demPrice.reset_index(drop=True, inplace=True)
            print(demPrice)
            if(demPrice.empty):
                df_collection[key].loc[index,'NormalizedPrice'] = -1
            else:
                demPrice = demPrice.loc[0,'AvgPrice']
                demPrice = float(demPrice)
                df_collection[key].loc[index,'NormalizedPrice'] = repPrice / (demPrice+repPrice)

       Date Contract  AvgPrice  NormalizedPrice
0  05/01/00      Rep       0.5                0
       Date Contract  AvgPrice  NormalizedPrice
0  05/01/00      Dem       0.5              0.5
       Date Contract  AvgPrice  NormalizedPrice
0  05/02/00      Rep     0.495              0.0
       Date Contract  AvgPrice  NormalizedPrice
0  05/02/00      Dem      0.51         0.507463
       Date Contract  AvgPrice  NormalizedPrice
0  05/03/00      Rep     0.492              0.0
       Date Contract  AvgPrice  NormalizedPrice
0  05/03/00      Dem     0.509         0.508492
       Date Contract  AvgPrice  NormalizedPrice
0  05/04/00      Rep      0.49              0.0
       Date Contract  AvgPrice  NormalizedPrice
0  05/04/00      Dem     0.511          0.51049
       Date Contract  AvgPrice  NormalizedPrice
0  05/05/00      Rep     0.478              0.0
       Date Contract  AvgPrice  NormalizedPrice
0  05/05/00      Dem     0.516         0.519115
       Date Contract  AvgPrice  Normaliz

0  07/28/00      Dem     0.486         0.487952
       Date Contract  AvgPrice  NormalizedPrice
0  07/29/00      Rep     0.541              0.0
       Date Contract  AvgPrice  NormalizedPrice
0  07/29/00      Dem      0.46          0.45954
Empty DataFrame
Columns: [Date, Contract, AvgPrice, NormalizedPrice]
Index: []
       Date Contract  AvgPrice  NormalizedPrice
0  07/31/00      Rep     0.527              0.0
       Date Contract  AvgPrice  NormalizedPrice
0  07/31/00      Dem      0.47         0.471414
       Date Contract  AvgPrice  NormalizedPrice
0  08/01/00      Rep     0.529                0
       Date Contract  AvgPrice  NormalizedPrice
0  08/01/00      Dem     0.472         0.471528
       Date Contract  AvgPrice  NormalizedPrice
0  08/02/00      Rep     0.523              0.0
       Date Contract  AvgPrice  NormalizedPrice
0  08/02/00      Dem     0.473           0.4749
       Date Contract  AvgPrice  NormalizedPrice
0  08/03/00      Rep     0.515              0.0
       Da

0  10/12/00      Dem     0.553         0.549156
       Date Contract  AvgPrice  NormalizedPrice
0  10/13/00      Rep     0.461              0.0
       Date Contract  AvgPrice  NormalizedPrice
0  10/13/00      Dem     0.544         0.541294
       Date Contract  AvgPrice  NormalizedPrice
0  10/14/00      Rep     0.479              0.0
       Date Contract  AvgPrice  NormalizedPrice
0  10/14/00      Dem      0.53         0.525273
       Date Contract  AvgPrice  NormalizedPrice
0  10/15/00      Rep     0.491              0.0
       Date Contract  AvgPrice  NormalizedPrice
0  10/15/00      Dem     0.513         0.510956
       Date Contract  AvgPrice  NormalizedPrice
0  10/16/00      Rep     0.523              0.0
       Date Contract  AvgPrice  NormalizedPrice
0  10/16/00      Dem     0.487         0.482178
       Date Contract  AvgPrice  NormalizedPrice
0  10/17/00      Rep     0.502              0.0
       Date Contract  AvgPrice  NormalizedPrice
0  10/17/00      Dem     0.502          

In [3]:
for key in df_collection:
    print(df_collection[key])

        Date Contract  AvgPrice  NormalizedPrice
0   05/01/00      Dem     0.500         0.500000
2   05/01/00      Rep     0.500         0.500000
3   05/02/00      Dem     0.510         0.507463
5   05/02/00      Rep     0.495         0.492537
6   05/03/00      Dem     0.509         0.508492
8   05/03/00      Rep     0.492         0.491508
9   05/04/00      Dem     0.511         0.510490
11  05/04/00      Rep     0.490         0.489510
12  05/05/00      Dem     0.516         0.519115
14  05/05/00      Rep     0.478         0.480885
15  05/06/00      Dem     0.514         0.511443
17  05/06/00      Rep     0.491         0.488557
20  05/07/00      Rep     0.484        -1.000000
21  05/08/00      Dem     0.512         0.515609
23  05/08/00      Rep     0.481         0.484391
24  05/09/00      Dem     0.510         0.510000
26  05/09/00      Rep     0.490         0.490000
27  05/10/00      Dem     0.500         0.499500
29  05/10/00      Rep     0.501         0.500500
30  05/11/00      De